In [1]:
"""make variations of input image"""
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(os.path.abspath("/raid/home/himanshus/experiments/repos/stable-diffusion/blip"))

import argparse, os, sys, glob
import PIL
import torch

# original_backward = torch.Tensor.backward

# def patched_backward(tensor, *args, **kwargs):
#     kwargs['allow_unused'] = True
#     original_backward(tensor, *args, **kwargs)

# torch.Tensor.backward = patched_backward
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange, repeat
from torchvision.utils import make_grid, save_image
from torch import autocast
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader,Dataset
from contextlib import nullcontext
import time
from pytorch_lightning import seed_everything
import robustbench
from robustbench.data import load_cifar10, load_imagenet
from robustbench.utils import load_model, clean_accuracy

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler
from scripts.custom_Dataset import CustomDataset
from IPython.display import display, clear_output
from custom_pgd import PGD
from blip.models.blip import blip_decoder

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# from PIL import Image
# import requests
# import torch
# import torchvision
# from torchvision import transforms
# from torch.utils.data import Dataset, DataLoader
# from torchvision.transforms.functional import InterpolationMode
# import matplotlib.pyplot as plt
# import numpy as np
# import foolbox as fb
# from robustbench.data import load_cifar10

In [2]:
class parser():
    def __init__(self):
        self.prompt = "a painting of a virus monster playing guitar"
        self.init_img = "outputs/img2img-samples/grid-0003.png" #path to the input image
        self.outdir = "../BLIP/experiment_1/imagenet/clean_test_generated_2048" #dir to write results to
        self.skip_grid = True
        self.skip_save = False
        self.ddim_steps = 50 #number of ddim sampling steps
        self.plms = False
        self.fixed_code = False
        self.ddim_eta = 0.0 #ddim eta (eta=0.0 corresponds to deterministic sampling
        self.n_iter = 1
        self.C = 4 #latent channels
        self.f = 8
        self.n_samples = 16 #batch size
        self.n_rows = 0
        self.scale = 5.0 #unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))
        self.strength = 0.5 #strength for noising/unnoising. 1.0 corresponds to full destruction of information in init image
        self.from_file = ""
        self.config = "configs/stable-diffusion/v1-inference.yaml"
        self.ckpt = "models/ldm/stable-diffusion-v1/model.ckpt"
        self.seed = 42
        self.precision = "autocast" #["full", "autocast"]
        

In [3]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)
    model.cuda()
    model.eval()
    return model


def load_img(path):
    image = Image.open(path).convert("RGB")
    w, h = image.size
    print(f"loaded input image of size ({w}, {h}) from {path}")
    w, h = map(lambda x: x - x % 64, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=PIL.Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.*image - 1.

In [4]:
from blip.models.blip import blip_decoder

image_size = 384
# image = load_demo_image(image_size=image_size, device=device)

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
model_path = "./blip/checkpoints/model_base_capfilt_large.pth"
    
blip_model = blip_decoder(pretrained=model_path, image_size=image_size, vit='base')
blip_model.eval()
blip_model = blip_model.to(device)

reshape position embedding from 196 to 576
load checkpoint from ./blip/checkpoints/model_base_capfilt_large.pth


In [5]:
opt = parser()
seed_everything(opt.seed)

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")

model = model.to(device)

Global seed set to 42


Loading model from models/ldm/stable-diffusion-v1/model.ckpt
Global Step: 840000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.6.self_attn.out_proj.weight', 'vision_model.encoder.layers.4.self_attn.k_proj.bias', 'vision_model.encoder.layers.13.self_attn.k_proj.weight', 'vision_model.encoder.layers.7.self_attn.q_proj.weight', 'vision_model.encoder.layers.15.self_attn.v_proj.bias', 'vision_model.encoder.layers.17.self_attn.k_proj.bias', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.8.mlp.fc2.weight', 'vision_model.encoder.layers.0.self_attn.out_proj.weight', 'vision_model.encoder.layers.4.self_attn.out_proj.bias', 'vision_model.encoder.layers.9.mlp.fc2.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.bias', 'vision_model.encoder.layers.16.self_attn.v_proj.bias', 'vision_model.encoder.layers.21.layer_norm1.bias', 'vision_model.encoder.layers.8.self_attn.out_proj.bias', 'vision_model.encoder.layers.16.self_attn.q_proj.bias

In [6]:
#added by himanshu for adding batch processing
blip_normalize = transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
# classifier_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
classifier_transform = transforms.Compose([transforms.ToTensor()])
                                # transforms.Resize((96,96)),])
                                # classifier_normalize])
transform_blip = transforms.Compose([transforms.Resize(size=(384,384)),
                                blip_normalize
                                ])
# classifier_inv_normalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
#                                                      std = [ 1/0.229, 1/0.224, 1/0.225 ]),
#                                 transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
#                                                      std = [ 1., 1., 1. ]),
#                                ])
blip_inv_normalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1./0.26862954, 1./0.26130258, 1./0.27577711 ]),
                                transforms.Normalize(mean = [ -0.48145466, -0.4578275, -0.40821073 ],
                                                     std = [ 1., 1., 1. ]),])
                                # transforms.Resize((32,32))
                            #    ])
inverse_resize = transforms.Compose([transforms.Resize((32,32))])
# inverse_resize_2 = transforms.Compose([transforms.Resize((32,32))])
batch_size = opt.n_samples
num_classes = 100
# image_folder = '../BLIP/experiment_1/cifar10/clean_test/'
# label_file = '../BLIP/experiment_1/cifar10/clean_test_labels.txt'
image_folder = '../BLIP/experiment_1/imagenet/clean_test_2048'
label_file = '../BLIP/experiment_1/imagenet/clean_test_labels_2048.txt'
# captions_file = "../BLIP/experiment_1/cifar10/clean_test_captions.txt"
# test_dataset = CustomDataset(image_folder, label_file, transform=classifier_transform)
# test_dataset = datasets.CIFAR100("../data/", train=False, transform=classifier_transform, download=True)
# train_dataset = datasets.STL10(root="../data",
#                            split="train",
#                            transform=transform,
#                           download=True)
# test_dataset = datasets.STL10(root="../data",
#                          split="test",
#                          transform=transform)

# train_loader = torch.utils.data.DataLoader(train_dataset,
#                                        batch_size=batch_size,
#                                        shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
#                                      shuffle=False, num_workers=8)

# class_names = train_dataset.classes
# data_iterator = iter(train_loader)
# batch = next(data_iterator)

In [7]:
if opt.plms:
    raise NotImplementedError("PLMS sampler not (yet) supported")
    sampler = PLMSSampler(model)
else:
    sampler = DDIMSampler(model)

os.makedirs(opt.outdir, exist_ok=True)
outpath = opt.outdir

batch_size = opt.n_samples
n_rows = opt.n_rows if opt.n_rows > 0 else batch_size

In [8]:
assert 0. <= opt.strength <= 1., 'can only work with strength in [0.0, 1.0]'
t_enc = int(opt.strength * opt.ddim_steps)
print(f"target t_enc is {t_enc} steps")

precision_scope = autocast if opt.precision == "autocast" else nullcontext

target t_enc is 25 steps


In [9]:
mean = [0.48145466, 0.4578275, 0.40821073]
std = [0.26862954, 0.26130258, 0.27577711]
def inverse_normalize(batch_tensor, mean=mean, std=std):
    for tensor, m, s in zip(batch_tensor, mean, std):
        tensor.mul_(s).add_(m)
    return batch_tensor

def defense(img):
    x_samples = None
    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                tic = time.time()
                # clear_output(wait=True)
                init_image = transform_blip(img).to(device)
                # data = [list(batch[2])]
                # print(data)
                data = [blip_model.generate(init_image, sample=False, num_beams=3, max_length=20, min_length=5)]
                # print(data)
                num = len(img)
                init_image = inverse_normalize(init_image)
                init_latent = model.get_first_stage_encoding(model.encode_first_stage(init_image))  # move to latent space
                sampler.make_schedule(ddim_num_steps=opt.ddim_steps, ddim_eta=opt.ddim_eta, verbose=False)
                for n in range(opt.n_iter):
                    for prompts in data:
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(num * [""])
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)
                        c = model.get_learned_conditioning(prompts)

                        # encode (scaled latent)
                        z_enc = sampler.stochastic_encode(init_latent, torch.tensor([t_enc]*num).to(device))
                        # decode it
                        samples = sampler.decode(z_enc, c, t_enc, unconditional_guidance_scale=opt.scale,
                                                unconditional_conditioning=uc,)

                        x_samples = model.decode_first_stage(samples)
                        x_samples = torch.clamp((x_samples + 1.0) / 2.0, min=0.0, max=1.0)

                toc = time.time()
    return inverse_resize(x_samples)

In [2]:
from tqdm import tqdm

mean_rgb = (0.4914, 0.4822, 0.4465)
std_rgb = (0.2023, 0.1994, 0.2010)
jitter_param = 0.4

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=mean_rgb,
#         std=std_rgb),
# ])

transform = transforms.Compose([
    transforms.ToTensor()])

def calculate_accuracy(model, dataloader):
    total_samples = 0
    correct_predictions = 0
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, dim=1)

            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = correct_predictions / total_samples
    return accuracy

In [12]:
test_dataset = datasets.CIFAR100(root="../data",train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, num_workers=8)

Files already downloaded and verified


In [12]:
from pytorchcv.model_provider import get_model as ptcv_get_model
# import torch
# from torch.autograd import Variable

class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean).view(-1, 1, 1)
        self.std = torch.Tensor(std).view(-1, 1, 1)

    def forward(self, x):
        if x.is_cuda:
            self.mean = self.mean.to(x.device)
            self.std = self.std.to(x.device)
        return (x - self.mean) / self.std
    
clf = ptcv_get_model("wrn28_10_cifar100", pretrained=True)
norm = Normalize(mean_rgb, std_rgb)   
clf = nn.Sequential(norm, clf)
clf.eval()

Sequential(
  (0): Normalize()
  (1): CIFARWRN(
    (features): Sequential(
      (init_block): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (stage1): Sequential(
        (unit1): PreResUnit(
          (body): PreResBlock(
            (conv1): PreConvBlock(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ): ReLU(inplace=True)
              (conv): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
            (conv2): PreConvBlock(
              (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ): ReLU(inplace=True)
              (conv): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
          )
          (identity_conv): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (unit2): PreResUnit(
          (b

In [13]:
calculate_accuracy(clf, test_loader)

  0%|          | 0/625 [00:00<?, ?it/s]

100%|██████████| 625/625 [00:05<00:00, 109.04it/s]


0.8213

In [14]:
import torchattacks
atk = torchattacks.PGD(clf, eps=8./255., alpha=2./255.)
atk.set_normalization_used(mean=mean_rgb, std=std_rgb)

In [15]:
clf = clf.to(device)
clf.eval()
correct = 0
total = 0

for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    adv_images = atk(images, labels)
    p_img = defense(adv_images)
    outputs = clf(defense(p_img))
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Robust Accuracy: {(100 * correct / total):.2f}%')

 10%|█         | 64/625 [18:40<2:43:44, 17.51s/it]


KeyboardInterrupt: 

In [9]:
class Normalize(nn.Module):
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.Tensor(mean).view(-1, 1, 1)
        self.std = torch.Tensor(std).view(-1, 1, 1)

    def forward(self, x):
        if x.is_cuda:
            self.mean = self.mean.to(x.device)
            self.std = self.std.to(x.device)
        return (x - self.mean) / self.std
norm = Normalize(mean_rgb, std_rgb)   
model = nn.Sequential(norm, model)

In [15]:
label = []
for i in test_loader:
    label.extend(i[1])

In [18]:
for i in label:
    print(i.item())

49
33
72
51
71
92
15
14
23
0
71
75
81
69
40
43
92
97
70
53
70
49
75
29
21
16
39
8
8
70
20
61
41
93
56
73
58
11
25
37
63
24
49
73
56
22
41
58
75
17
4
6
9
57
2
32
71
52
42
69
77
27
15
65
7
35
43
82
63
92
66
90
67
91
32
32
82
10
77
22
71
78
54
6
29
89
78
33
11
67
22
18
27
21
13
21
50
75
37
35
26
83
47
95
43
69
76
17
57
59
25
20
27
0
9
71
8
43
57
56
85
10
19
92
33
20
21
50
70
46
11
16
1
74
33
91
60
64
52
23
4
11
52
37
24
95
25
39
51
58
58
77
18
59
45
66
58
20
24
4
36
8
87
10
30
47
54
99
51
83
9
37
4
83
95
83
32
73
18
40
39
64
22
80
28
28
40
95
98
83
12
24
45
13
94
24
58
63
7
87
6
78
68
60
6
23
44
31
80
93
73
98
49
90
97
59
2
67
16
81
94
27
76
77
12
18
0
76
79
71
89
57
47
24
65
0
32
36
82
31
23
24
34
21
11
53
80
44
4
39
91
16
36
68
50
97
58
31
6
42
80
76
89
55
19
91
70
1
6
62
99
51
96
83
42
18
67
66
40
62
78
84
28
89
30
66
18
38
42
92
27
11
86
44
96
12
16
67
43
89
96
12
40
7
86
77
76
31
18
28
19
18
41
42
43
31
93
15
10
8
37
89
32
67
12
2
19
91
94
7
71
36
61
62
5
60
45
91
34
95
9
40
68
62
99

In [ ]:
"../BLIP/experiment_1/cifar100/clean_train_generated/{}.png"